In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
 # Data already downloaded 
# let's open the file
fileIN = '../jet_notebooks/Data-MLtutorial/JetDataset/jetImage_7_100p_30000_40000.h5'
f = h5py.File(fileIN)
# and see what it contains
print(list(f.keys()))

In [ ]:
# These are the quantities we are dealing with
featurenames = f.get('jetFeatureNames')
print(featurenames[:])
# the b is due to the byte vs utf-8 encoding of the strings in the dataset
# just ignore them for the moment

In [ ]:
print(f.get('jetImage'))

In [ ]:
jet_data = np.array(f.get('jets'))
target = jet_data[:,-6:-1]
# shape of the dataset
print("Dataset shape:")
print(target.shape)
print("First five entries:")
for i in range(5):
    print(target[i])
print("Last 5 entries:")
for i in range(-5,0):
    print(target[i])

In [ ]:
data = np.array(jet_data[:,:-6])
print(data.shape)

In [ ]:
labelCat= ["gluon", "quark", "W", "Z", "top"]
# this function makes the histogram of a given quantity for the five classes
def makePlot(feature_index, input_data, input_featurenames):
    plt.subplots()
    for i in range(len(labelCat)):
        # notice the use of numpy masking to select specific classes of jets
        my_data = input_data[np.argmax(target, axis=1) == i]
        # then plot the right quantity for the reduced array
        plt.hist(my_data[:,feature_index], 50, density=True, histtype='step', fill=False, linewidth=1.5)
    plt.yscale('log')    
    plt.legend(labelCat, fontsize=12, frameon=False)
    plt.xlabel(str(input_featurenames[feature_index], "utf-8"), fontsize=15)
    plt.ylabel('Prob. Density (a.u.)', fontsize=15)
    plt.show()
    #del fig, ax
    #return fig, ax

In [ ]:
# we now plot all the features
for i in range(len(featurenames[:-6])):
    makePlot(i, data, featurenames)
    #fig.show()

In [ ]:
from matplotlib.colors import LogNorm
labelCat= ["gluon", "quark", "W", "Z", "top"]
image = np.array(f.get('jetImage'))
image_g = image[np.argmax(target, axis=1) == 0]
image_q = image[np.argmax(target, axis=1) == 1]
image_W = image[np.argmax(target, axis=1) == 2]
image_Z = image[np.argmax(target, axis=1) == 3]
image_t = image[np.argmax(target, axis=1) == 4]
images = [image_q, image_g, image_W, image_Z, image_t]
#plt.rc('text', usetex=True) #you can uncomment this if you have a latex installation
plt.rc('font', family='serif')
for i in range(len(images)):
    SUM_Image = np.sum(images[i], axis = 0)
    plt.imshow(SUM_Image/float(images[i].shape[0]), origin='lower',norm=LogNorm(vmin=0.01))
    plt.colorbar()
    plt.title(labelCat[i], fontsize=15)
    plt.xlabel("Delta_eta cell", fontsize=15)
    plt.ylabel("Delta_phi cell", fontsize=15)
    plt.show()

In [ ]:
p_featurenames = f.get("particleFeatureNames")
print(p_featurenames[:])

In [ ]:
p_data = f.get("jetConstituentList")
print(p_data.shape)

In [ ]:
labelCat= ["gluon", "quark", "W", "Z", "top"]
# this function makes the histogram of a given quantity for the five classes
def makePlot_p(feature_index, input_data, input_featurenames):
    plt.subplots()
    for i in range(len(labelCat)):
        my_data = input_data[:,:,feature_index]
        # notice the use of numpy masking to select specific classes of jets
        my_data = my_data[np.argmax(target, axis=1) == i]
        # then plot the right quantity for the reduced array
        plt.hist(my_data[:,feature_index].flatten(), 50, density=True, histtype='step', fill=False, linewidth=1.5)
    plt.yscale('log')    
    plt.legend(labelCat, fontsize=12, frameon=False)  
    plt.xlabel(str(input_featurenames[feature_index], "utf-8"), fontsize=15)
    plt.ylabel('Prob. Density (a.u.)', fontsize=15)
    plt.show()
    #del fig, ax
    #return fig, ax

In [ ]:
# we now plot all the features
for i in range(len(p_featurenames)-1):
    makePlot_p(i, p_data, p_featurenames)
    #fig.show()

In [ ]:
import keras
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

# Importing keras and related modules
import tensorflow as tf
from keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from keras.layers import Input,Dense,Flatten,Dropout,Reshape,Conv2D,MaxPooling2D,UpSampling2D,Conv2DTranspose
from tensorflow.keras.layers import BatchNormalization
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adadelta, RMSprop,SGD,Adam
from keras import regularizers
from keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, Lambda
from tensorflow.keras.losses import mse


In [ ]:
# Prepare the data
# Assuming you have jet images in a numpy array 'jet_images'
# Normalize the pixel values between 0 and 1
jet_images = np.array(f.get('jetImage')).astype('float32') / 255.0


In [ ]:
# Get the shape of the first image in the dataset
first_image_shape = jet_images[0].shape

# Print the dimensions of the first image
print("Dimensions of the first image:", first_image_shape)

In [ ]:
dataset_shape = jet_images.shape
# Print the shape of the dataset
print("Shape of the dataset:", dataset_shape)
image_size = dataset_shape[1]
print(f'image_size is,',image_size)

In [ ]:
# Split the data into training and validation sets
train_size = int(0.8 * len(jet_images))
x_train = jet_images[:train_size]
x_val = jet_images[train_size:]


In [ ]:
# Add the batch dimension to your training and validation data
x_train = np.expand_dims(x_train, axis=-1)  # Add a single channel dimension
x_val = np.expand_dims(x_val, axis=-1)

In [ ]:
# VAE architecture
latent_dim = 2  # Dimension of latent space

def build_vae(input_shape):
    # Encoder
    encoder_input = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoder_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = Flatten()(x)
    z_mean = Dense(latent_dim)(x)
    z_log_var = Dense(latent_dim)(x)
    
    # Reparameterization trick
    def sampling(args):
        z_mean, z_log_var = args
        epsilon = tf.random.normal(shape=(tf.shape(z_mean)[0], latent_dim), mean=0., stddev=1.)
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon
    
    z = Lambda(sampling)([z_mean, z_log_var])
    
    encoder = Model(encoder_input, [z_mean, z_log_var, z], name='encoder')
    
    # Decoder
    latent_input = Input(shape=(latent_dim,))
    x = Dense(image_size // 4 * image_size // 4 * 64, activation='relu')(latent_input)
    x = Reshape((image_size // 4, image_size // 4, 64))(x)
    x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
    x = Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
    decoder_output = Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)
    
    decoder = Model(latent_input, decoder_output, name='decoder')
    
    # VAE model
    _, _, encoded = encoder(encoder_input)
    vae_output = decoder(encoded)
    
    vae = Model(encoder_input, vae_output, name='vae')
    
    # VAE loss
    reconstruction_loss = mse(tf.reshape(encoder_input, [-1, image_size * image_size]), tf.reshape(vae_output, [-1, image_size * image_size]))
    kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
    kl_loss = tf.reduce_sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = tf.reduce_mean(reconstruction_loss + kl_loss)
    
    vae.add_loss(vae_loss)
    
    return vae, encoder, decoder



In [ ]:
# Reshape the jet images dataset to match the model's input shape
jet_images_reshaped = jet_images.reshape((-1, image_size, image_size, 1))

# Build and compile the VAE
input_shape = jet_images_reshaped[0].shape
vae, encoder, decoder = build_vae(input_shape)
vae.compile(optimizer='adam')


# ...



In [ ]:
# Training loop
epochs = 50
batch_size = 4  # Set the batch size based on dataset size

for epoch in range(epochs):
    for _ in range(jet_images_reshaped.shape[0] // batch_size):
        batch_indices = np.random.randint(0, jet_images_reshaped.shape[0], batch_size)
        batch = jet_images_reshaped[batch_indices]
        vae.train_on_batch(batch, batch)
    
    print(f"Epoch {epoch}/{epochs}, Loss: {vae.evaluate(jet_images_reshaped, jet_images_reshaped, verbose=0)}")

# Generate and save example images from the decoder
num_examples = 10
latent_samples = np.random.normal(size=(num_examples, latent_dim))
generated_images = decoder.predict(latent_samples)

# Display or save generated images